# Fino uravnavanje Open AI modelov

Ta zvezek temelji na trenutnih navodilih iz dokumentacije [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) podjetja Open AI.

Fino uravnavanje izboljša delovanje osnovnih modelov za vašo aplikacijo tako, da jih ponovno izurite z dodatnimi podatki in kontekstom, ki so pomembni za vaš primer uporabe ali scenarij. Upoštevajte, da tehnike oblikovanja pozivov, kot sta _few shot learning_ in _retrieval augmented generation_, omogočajo izboljšanje privzetega poziva z ustreznimi podatki za boljšo kakovost. Vendar pa so te metode omejene z največjo velikostjo okna za žetone, ki jo podpira izbrani osnovni model.

S finim uravnavanjem dejansko ponovno izurimo sam model z zahtevanimi podatki (kar nam omogoča uporabo veliko več primerov, kot jih lahko vključimo v največje okno za žetone) in uvedemo _prilagojeno_ različico modela, ki pri sklepanju ne potrebuje več primerov. To ne le izboljša učinkovitost oblikovanja pozivov (imamo več svobode pri uporabi okna za žetone za druge stvari), ampak lahko tudi zmanjša stroške (ker moramo modelu ob sklepanju poslati manj žetonov).

Fino uravnavanje poteka v 4 korakih:
1. Pripravite učne podatke in jih naložite.
1. Zaženite učno opravilo in pridobite fino uravnan model.
1. Ocenite fino uravnan model in po potrebi izboljšajte kakovost.
1. Ko ste zadovoljni, uvedite fino uravnan model za sklepanja.

Upoštevajte, da vsi osnovni modeli ne podpirajo finega uravnavanja – [preverite dokumentacijo OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) za najnovejše informacije. Prav tako lahko fino uravnate že prej fino uravnan model. V tej vadnici bomo za fino uravnavanje uporabili model `gpt-35-turbo` kot naš osnovni model.

---


### Korak 1.1: Pripravite svoj podatkovni niz

Zgradimo klepetalnika, ki vam pomaga razumeti periodni sistem elementov tako, da na vprašanja o elementu odgovarja z limerikom. V _tem_ preprostem vodiču bomo ustvarili podatkovni niz za učenje modela z nekaj vzorčnimi primeri odgovorov, ki prikazujejo pričakovano obliko podatkov. V resnični uporabi bi morali pripraviti podatkovni niz z veliko več primeri. Če obstaja odprt podatkovni niz (za vaše področje uporabe), ga lahko uporabite in preoblikujete za uporabo pri dodatnem učenju.

Ker se osredotočamo na `gpt-35-turbo` in želimo enkratni odgovor (chat completion), lahko pripravimo primere po [tem predlaganem formatu](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), ki ustreza zahtevam za OpenAI chat completion. Če pričakujete večkrožni pogovor, uporabite [format za večkrožne primere](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), ki vključuje parameter `weight` za označevanje, katere sporočila naj se uporabijo (ali ne) pri dodatnem učenju.

Za ta vodič bomo uporabili enostavnejši enokrožni format. Podatki so v [jsonl formatu](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), kjer je vsak zapis v eni vrstici, predstavljen kot objekt v JSON obliki. Spodnji izsek prikazuje 2 zapisa kot primer – celoten vzorčni niz (10 primerov), ki ga bomo uporabili za naš vodič o dodatnem učenju, si lahko ogledate v [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl). **Opomba:** Vsak zapis _mora_ biti zapisan v eni vrstici (ne razdeljen čez več vrstic, kot je običajno v formatiranih JSON datotekah)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

V resnični uporabi boste za dobre rezultate potrebovali veliko večji nabor primerov – kompromis je med kakovostjo odgovorov in časom/stroški dodatnega učenja. Uporabljamo majhen nabor, da lahko hitro zaključimo postopek in ga prikažemo. Za bolj zahteven vodič o dodatnem učenju si oglejte [ta primer iz OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst).


### Korak 1.2 Naložite svoj podatkovni niz

Podatke naložite z uporabo Files API [kot je opisano tukaj](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Upoštevajte, da morate pred zagonom te kode najprej opraviti naslednje korake:
 - Namestiti morate Python paket `openai` (prepričajte se, da uporabljate različico >=0.28.0 zaradi najnovejših funkcij)
 - Nastaviti morate okoljsko spremenljivko `OPENAI_API_KEY` na vaš OpenAI API ključ
Za več informacij si oglejte [navodila za nastavitev](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), ki so priložena temu tečaju.

Sedaj zaženite kodo, da ustvarite datoteko za nalaganje iz vaše lokalne JSONL datoteke.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Korak 2.1: Ustvarite nalogo za fino nastavljanje s pomočjo SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Korak 2.2: Preverite stanje opravila

Tukaj je nekaj stvari, ki jih lahko naredite z API-jem `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Prikaže zadnjih n opravil za fino nastavljanje
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Pridobi podrobnosti določenega opravila za fino nastavljanje
- `client.fine_tuning.jobs.cancel(<job_id>)` - Prekliči opravilo za fino nastavljanje
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Prikaže do n dogodkov iz opravila
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prvi korak v postopku je _preverjanje učne datoteke_, da zagotovite, da so podatki v pravilni obliki.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Korak 2.3: Spremljajte dogodke za nadzor napredka


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Korak 2.4: Oglejte si stanje na nadzorni plošči OpenAI


Status lahko preverite tudi tako, da obiščete spletno stran OpenAI in raziščete razdelek _Fine-tuning_ na platformi. Tam boste videli status trenutne naloge in lahko spremljate zgodovino preteklih izvajanj. Na tem posnetku zaslona lahko vidite, da je prejšnje izvajanje spodletelo, drugo pa je bilo uspešno. Za boljše razumevanje: do tega je prišlo, ker je bila pri prvem zagonu uporabljena JSON datoteka z napačno oblikovanimi zapisi – ko je bila napaka odpravljena, je drugi zagon uspešno zaključen in model je postal na voljo za uporabo.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.sl.png)


Statusna sporočila in metrike si lahko ogledate tudi tako, da se pomaknete nižje po vizualni nadzorni plošči, kot je prikazano:

| Sporočila | Metrike |
|:---|:---|
| ![Sporočila](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.sl.png) |  ![Metrike](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.sl.png)|


### Korak 3.1: Pridobite ID in preizkusite fino nastavljeni model v kodi


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Korak 3.2: Naložite in preizkusite fino prilagojen model v Playgroundu

Fino prilagojen model lahko zdaj preizkusite na dva načina. Prvi način je, da obiščete Playground in v spustnem meniju Models izberete svoj novo fino prilagojen model med ponujenimi možnostmi. Druga možnost je, da uporabite možnost "Playground", ki je prikazana v plošči za fino prilagajanje (glejte zgornji posnetek zaslona), kar odpre _primerjalni_ pogled, kjer sta osnovni in fino prilagojeni model prikazana drug ob drugem za hitro oceno.

Preprosto vnesite sistemski kontekst, ki ste ga uporabili v svojih učnih podatkih, in dodajte svoje testno vprašanje. Opazili boste, da se na obeh straneh posodobita enak kontekst in vprašanje. Zaženite primerjavo in videli boste razliko v izhodih med obema modeloma. _Opazite, kako fino prilagojeni model oblikuje odgovor v formatu, ki ste ga določili v svojih primerih, medtem ko osnovni model zgolj sledi sistemskemu pozivu_.

Opazili boste tudi, da primerjava prikaže število žetonov za vsak model in čas, potreben za sklepanje. **Ta primer je zelo poenostavljen in je namenjen prikazu postopka, ne pa odražanju resničnega nabora podatkov ali scenarija**. Morda boste opazili, da imata oba primera enako število žetonov (sistemski kontekst in uporabniški poziv sta enaka), pri čemer fino prilagojeni model za sklepanje porabi več časa (ker gre za prilagojen model).

V resničnih primerih ne boste uporabljali takšnega preprostega primera, temveč boste model fino prilagajali na resničnih podatkih (npr. katalog izdelkov za podporo strankam), kjer bo kakovost odgovora veliko bolj očitna. V _takšnem_ kontekstu boste za enakovredno kakovost odgovora z osnovnim modelom potrebovali več prilagajanja pozivov, kar bo povečalo porabo žetonov in verjetno tudi čas obdelave za sklepanje. _Če želite to preizkusiti, si oglejte primere fine-tuninga v OpenAI Cookbook in začnite._



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko samodejni prevodi vsebujejo napake ali netočnosti. Izvirni dokument v svojem izvirnem jeziku naj velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazume ali napačne razlage, ki bi izhajale iz uporabe tega prevoda.
